In [1]:
import pandas as pd
import numpy as np
from PIL import Image
import os
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import shap

c:\Users\rkdeo\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_fdc = pd.read_excel('./data/raw/Sensor_Data_DSS_반출.xlsx')

동일값으로만 이루어진 열 확인

In [3]:
def find_constant_columns(df):
    constant_columns = {}
    for col in df.columns:
        if df[col].nunique() == 1:  
            constant_columns[col] = df[col].iloc[0] 
    return constant_columns

constant_columns = find_constant_columns(df_fdc)
print("같은 값으로 이루어진 열과 값:", constant_columns)

같은 값으로 이루어진 열과 값: {}


결측치 확인

In [4]:
def count_all_nan_rows(df):
    nan_rows_count = df.iloc[:, 5:].isna().all(axis=1).sum()
    print(f"모든 값이 NaN인 행의 개수: {nan_rows_count}")

count_all_nan_rows(df_fdc)

모든 값이 NaN인 행의 개수: 243


In [5]:
def remove_all_nan_rows(df):
    nan_rows = df.iloc[:, 5:].isna().all(axis=1)
    df_cleaned = df[~nan_rows]
    return df_cleaned

df_fdc_remove_missing = remove_all_nan_rows(df_fdc)
print(f'{df_fdc.shape} → {df_fdc_remove_missing.shape}')

(1309, 276) → (1066, 276)


In [6]:
count_all_nan_rows(df_fdc_remove_missing)

모든 값이 NaN인 행의 개수: 0


In [23]:
def get_columns_with_missing_values(df):
    missing_values = df.isnull().sum()  
    missing_columns = missing_values[missing_values > 0]
    return missing_columns

missing_columns = get_columns_with_missing_values(df_fdc_remove_missing)

In [8]:
def remove_columns_with_missing_values(df, missing_columns):
    df_cleaned = df.drop(columns=missing_columns.index)
    return df_cleaned

df_fdc_cleaned = remove_columns_with_missing_values(df_fdc_remove_missing, missing_columns)

제거 후 남은 열: Index(['ID', 'EQUIP_ID', 'DATE_TIME', 'YVALUE', 'PASS(1)/FAIL(0)', 'X38_05',
       'X30_10', 'X34_02', 'X53_02', 'X52_10',
       ...
       'X39_13', 'X41_13', 'X50_13', 'X53_13', 'X57_13', 'X58_13.1', 'X36_13',
       'X45_13', 'X51_13', 'X34_13'],
      dtype='object', length=221)


In [9]:
no_image_list = ['ID_1231', 'ID_1233', 'ID_1239', 'ID_1241', 'ID_1244', 'ID_1245','ID_1309']
df_final = df_fdc_cleaned[~df_fdc_cleaned['ID'].isin(no_image_list)]

정규화

In [10]:
scaler =StandardScaler()

sensor_data = df_final.iloc[:,5:]
sensor_data_scaled = scaler.fit_transform(sensor_data)
df_sensor_scaled = pd.DataFrame(sensor_data_scaled, columns=sensor_data.columns)
df_fdc_scaled = pd.concat([df_final.iloc[:,:5].reset_index(drop=True), df_sensor_scaled.reset_index(drop=True)], axis=1)

라벨 추출

In [11]:
labels = df_fdc_scaled['PASS(1)/FAIL(0)'].values
labels = np.array(labels)

이미지 데이터 추출 및 배열 변환

In [12]:
def get_image_filenames(image_folder, image_ids):
    image_filenames = []
    for image_id in image_ids:
        image_path = os.path.join(image_folder, f"{image_id}.png")
        if os.path.exists(image_path): 
            image_filenames.append(image_path) 
        else:
            print(f"파일을 찾을 수 없습니다: {image_id}")
    return image_filenames

image_folder = './data/image'
image_ids = df_final['ID']
image_filenames = get_image_filenames(image_folder, image_ids)

In [22]:
def compare_image_order():
    image_ids = [os.path.basename(filename).split('.')[0] for filename in image_filenames]
    comparison_result = df_final['ID'].astype(str) == image_ids

    if comparison_result.all():
        print("image_filenames의 순서와 df_final['ID']의 순서가 모두 일치합니다.")
    else:
        print("일치하지 않는 항목이 있습니다.")
        mismatch_indices = comparison_result[comparison_result == False].index
        print(f"일치하지 않는 인덱스: {mismatch_indices.tolist()}")

compare_image_order()

image_filenames의 순서와 df_final['ID']의 순서가 모두 일치합니다.


In [17]:
def image_to_numpy(file_path):
    image = Image.open(file_path).convert('L')  # Convert image to grayscale
    image = image.resize((128, 128))
    return np.array(image) / 255.0

image_data = np.array([image_to_numpy(file) for file in image_filenames])

Train, Validation, Test

In [18]:
X_fdc_temp, X_fdc_val, X_img_temp, X_img_val, y_temp, y_val = train_test_split(
    df_sensor_scaled, image_data, labels, test_size=0.2, random_state=42
)

X_fdc_train, X_fdc_test, X_img_train, X_img_test, y_train, y_test = train_test_split(
    X_fdc_temp, X_img_temp, y_temp, test_size=0.25, random_state=42
)  # 여기서 test_size=0.25는 validation+test의 20% 중 25%를 test로

모델

1. FDC data → Image 예측

2. Image → Scratch 분류

1. 조건부 생성 모델(Conditional Generative Models)
조건부 생성적 적대 신경망(Conditional GAN, cGAN):

개념: 숫자 데이터를 조건(condition)으로 받아 이미지를 생성하는 모델입니다.
장점: 입력 데이터의 변동이 작더라도, 생성기(Generator)가 다양한 이미지를 생성할 수 있습니다.
구성:
생성기(Generator): 숫자 데이터와 랜덤 노이즈를 입력으로 받아 이미지를 생성합니다.
판별기(Discriminator): 실제 이미지와 생성된 이미지를 구분하며, 이때 숫자 데이터를 함께 입력으로 받습니다.
조건부 변분 오토인코더(Conditional Variational Autoencoder, CVAE):

개념: 숫자 데이터를 조건으로 사용하여 이미지를 생성하는 오토인코더 구조입니다.
장점: 잠재 공간(Latent Space)을 활용하여 데이터의 다양성을 표현할 수 있습니다.

3. 멀티모달 딥러닝(Multimodal Deep Learning)
추가 입력 데이터 활용:
만약 가능하다면, 이미지에 영향을 줄 수 있는 다른 변수나 센서 데이터 등을 추가로 수집하여 모델의 입력으로 사용합니다.
모달리티 간의 상호작용 학습:
숫자 데이터와 다른 형태의 데이터를 함께 입력하여 모델이 복잡한 패턴을 학습하도록 합니다.
4. 주의 메커니즘(Attention Mechanism) 도입
Self-Attention 또는 Cross-Attention:
모델이 입력 데이터의 중요한 부분에 집중하여 출력 이미지를 생성하도록 합니다.
Transformers 기반 모델:
GPT나 Vision Transformer(ViT) 등을 활용하여 이미지 생성에 적용할 수 있습니다.
6. 중간 표현 학습
Autoencoder를 통한 특성 추출:
숫자 데이터를 입력으로 중간 표현을 학습하고, 이를 기반으로 이미지를 생성합니다.
Decoder 네트워크 사용:
중간 표현에서 이미지를 복원하거나 생성하는 디코더를 설계합니다.